# Boltzmann Machine



## 一、Boltzmann Machine 简介
![](./image/2.png)

**Boltzmann Machine（BM）**是一种随机生成模型，由 Geoffrey Hinton 等人在 1985 年提出。它是一种**能量基模型（Energy-Based Model, EBM）**，可以建模联合概率分布，具有强大的表示能力，广泛应用于图像建模、特征学习、无监督学习等。

Boltzmann Machine 是 Hopfield 网络的概率版本，其能量函数和神经元状态都是**随机变量**。

---

## 二、基本结构与表示
![](./image/3.png)

一个标准的 Boltzmann Machine 包含：

* 一组**可见单元** $\mathbf{v} \in \{0, 1\}^n$
* 一组**隐藏单元** $\mathbf{h} \in \{0, 1\}^m$

两种节点组成的网络通常为**对称结构**，连接权重构成一个**对称矩阵** $W$，包含：

* $W_{ij}$ ：表示单元 $i$ 与 $j$ 之间的权重
* $b_i$ ：表示每个单元的偏置（bias）

> 如果所有单元之间都可以连接（包括隐藏–隐藏，显–显），叫做**通用 Boltzmann Machine**；
> 如果仅允许**显–隐之间连接**，而禁止显–显和隐–隐连接，就叫做**受限玻尔兹曼机（Restricted Boltzmann Machine, RBM）**，后者是更常见的形式。

---

## 三、能量函数定义

对于给定的状态 $(\mathbf{v}, \mathbf{h})$，其能量函数定义为：

$$
E(\mathbf{v}, \mathbf{h}) = - \sum_{i,j} W_{ij} x_i x_j - \sum_i b_i x_i
$$

其中，$x_i \in \mathbf{v} \cup \mathbf{h}$。如果使用 RBM 结构（无自连接，分层结构），能量函数可简化为：

$$
E(\mathbf{v}, \mathbf{h}) = - \sum_{i=1}^{n} \sum_{j=1}^{m} v_i W_{ij} h_j - \sum_{i=1}^{n} b_i v_i - \sum_{j=1}^{m} c_j h_j
$$

---

## 四、概率建模：Gibbs 分布

Boltzmann Machine 通过能量函数定义一个**联合概率分布**：

$$
P(\mathbf{v}, \mathbf{h}) = \frac{1}{Z} \exp(-E(\mathbf{v}, \mathbf{h}))
$$

其中，归一化因子 $Z$ 为配分函数（partition function）：

$$
Z = \sum_{\mathbf{v}, \mathbf{h}} \exp(-E(\mathbf{v}, \mathbf{h}))
$$

而可见变量的边缘概率为：

$$
P(\mathbf{v}) = \sum_{\mathbf{h}} P(\mathbf{v}, \mathbf{h})
$$

由于隐藏变量带来指数级组合，Z 极难计算，因此训练过程往往依赖采样近似。

---

## 五、训练方法：最大化似然 + 对比散度（Contrastive Divergence）

目标是最大化训练数据在模型下的对数似然：

$$
\log P(\mathbf{v}) = \log \sum_{\mathbf{h}} \exp(-E(\mathbf{v}, \mathbf{h})) - \log Z
$$

其梯度为：

$$
\frac{\partial \log P(\mathbf{v})}{\partial \theta} = \mathbb{E}_{P(\mathbf{h} | \mathbf{v})} \left[ \frac{\partial E(\mathbf{v}, \mathbf{h})}{\partial \theta} \right] - \mathbb{E}_{P(\mathbf{v}, \mathbf{h})} \left[ \frac{\partial E(\mathbf{v}, \mathbf{h})}{\partial \theta} \right]
$$

前一项是数据期望，后一项是模型期望。

由于模型期望极难精确计算，采用近似算法：

### Contrastive Divergence (CD-k)：

* 从数据点 $\mathbf{v}^{(0)}$ 出发
* Gibbs 采样 k 步，得到 $\mathbf{v}^{(k)}$
* 用 $\mathbf{v}^{(0)}$ 和 $\mathbf{v}^{(k)}$ 构造近似梯度

---

## 六、受限玻尔兹曼机（RBM）特点

为了提高训练效率，RBM 引入结构约束：

* 可见单元和隐藏单元之间是**双向全连接**
* 同一层内部没有连接

这一结构带来一个重要性质：

$$
P(h_j = 1 | \mathbf{v}) = \sigma\left( \sum_i W_{ij} v_i + c_j \right)
$$

$$
P(v_i = 1 | \mathbf{h}) = \sigma\left( \sum_j W_{ij} h_j + b_i \right)
$$

其中 $\sigma$ 是 Sigmoid 函数。这种**条件独立性**使得 Gibbs 采样非常高效。

---


## 应用与拓展

1. **Deep Belief Network (DBN)**：多个 RBM 堆叠组成深层生成模型
2. **Deep Boltzmann Machine (DBM)**：多层对称连接结构，更复杂的联合建模
3. **AE 与 RBM 区别**：AE 是前馈网络，RBM 是概率模型
4. **与 GAN/VAE 对比**：RBM 是显式建模，GAN 是隐式建模，VAE 是变分近似建模




In [ ]:
import torch
import torch.nn as nn

class RBM(nn.Module):
    def __init__(self, n_visible, n_hidden):
        super().__init__()
        # 初始化权重矩阵W（可见层到隐藏层）：维度 [n_visible, n_hidden]
        self.W = nn.Parameter(torch.randn(n_visible, n_hidden) * 0.01)

        # 可见层的偏置向量 bv，维度 [n_visible]
        self.bv = nn.Parameter(torch.zeros(n_visible))

        # 隐藏层的偏置向量 bh，维度 [n_hidden]
        self.bh = nn.Parameter(torch.zeros(n_hidden))

    def sample_h(self, v):
        """
        给定可见层v，采样隐藏层h
        返回值：
            - 概率 p(h=1 | v)
            - 按照该概率伯努利采样的h值（0或1）
        """
        prob_h = torch.sigmoid(v @ self.W + self.bh)  # 矩阵乘法 + 偏置，然后过sigmoid激活
        h_sample = torch.bernoulli(prob_h)            # 以概率为基础进行采样
        return prob_h, h_sample

    def sample_v(self, h):
        """
        给定隐藏层h，采样可见层v
        与 sample_h 类似
        """
        prob_v = torch.sigmoid(h @ self.W.t() + self.bv)  # 注意此处使用 W 的转置
        v_sample = torch.bernoulli(prob_v)
        return prob_v, v_sample

    def gibbs_sampling(self, v0, k=1):
        """
        执行k步Gibbs采样：v -> h -> v -> h -> ...
        参数：
            - v0: 初始可见层输入
            - k: 采样步数
        返回：
            - 最后一步采样得到的v_k
        """
        v = v0
        for _ in range(k):
            _, h = self.sample_h(v)
            _, v = self.sample_v(h)
        return v

    def contrastive_divergence(self, v0, lr=0.1, k=1):
        """
        使用 Contrastive Divergence（CD-k）算法更新参数
        参数：
            - v0: 输入数据（batch_size, n_visible）
            - lr: 学习率
            - k: Gibbs采样步数（CD-k）
        """
        # 正向传播：从 v0 得到隐藏层样本
        ph0, h0 = self.sample_h(v0)  # 初始隐藏层激活

        # 负向采样：进行 k 步 Gibbs 采样，得到模型的重构样本 vk
        vk = self.gibbs_sampling(v0, k)
        phk, hk = self.sample_h(vk)

        # 估计梯度（正负期望差）
        # 权重的更新：数据期望 - 模型期望
        W_grad = (v0.T @ ph0 - vk.T @ phk) / v0.size(0)
        bv_grad = (v0 - vk).mean(dim=0)
        bh_grad = (ph0 - phk).mean(dim=0)

        # 手动梯度更新（也可以用优化器）
        self.W.data += lr * W_grad
        self.bv.data += lr * bv_grad
        self.bh.data += lr * bh_grad
